# Imports

In [ ]:
from pathlib import Path

import keras

from matplotlib import pyplot as plt


## In colab

In [ ]:
from google.colab import userdata

In [ ]:
kaggle_target_path = Path('/root/.kaggle/kaggle.json')
kaggle_target_path.parent.mkdir(exist_ok=True, parents=True)
with open(kaggle_target_path,'w') as f:
  f.write(userdata.get('Kaggle'))

In [ ]:
! kaggle datasets download -d marquis03/fruits-100

In [ ]:
! unzip fruits-100.zip -d fruits-100

In [ ]:
! rm -r fruits-100.zip

In [ ]:
data_root_path = Path('fruits-100')
model_path = Path('haha.keras')

## In pc

In [ ]:
import yaml

In [ ]:
with open('local_paths.yaml') as f:
    local_paths = yaml.safe_load(f)

In [ ]:
data_root_path = Path(local_paths['data_root_path'])
model_path = Path(local_paths['data_root_path'])

## Loading train and validation data

In [ ]:
train_data_path = data_root_path / 'train'
val_data_path = data_root_path / 'val'

In [ ]:
train_data = keras.utils.image_dataset_from_directory(
    train_data_path, image_size=(150, 150))

val_data = keras.utils.image_dataset_from_directory(
    val_data_path, image_size=(150, 150))

In [ ]:
train_data_iterator = train_data.as_numpy_iterator()
k1 = next(train_data_iterator)

print(k1[1][0])
plt.imshow(k1[0][0].astype('uint8'))
plt.show()

In [ ]:
base_model = keras.applications.MobileNet(include_top=False)

base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))

x = base_model(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(100, activation="softmax")(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

## Compile the model

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Fit the model

In [ ]:
model.fit(train_data, epochs=20, validation_data=val_data)

## Save the model

In [ ]:
model.save(model_path)